## Shapefile Converter
A tool to convert Telkomsel's shapefile into GeoJSON format.    

**Writer** : ngakan.gandhi@dsanalytics.com  
**License** : MIT License.

In [1]:
# Import modules
import pandas as pd
import geopandas as gpd
import shapely
from pathlib import Path

In [2]:
# Set the path towards our data directory
data_dir = str(Path.cwd()) + '/PETA_V21'
out_dir = str(Path.cwd()) + '/results'

In [3]:
def load_shapefile_into_geojson(shp_in, geojson_out):
    """Load the .shp file and return as
    a GeoJSON file.
    """
    gdf = gpd.read_file(shp_in)
    gdf.to_file(geojson_out, driver='GeoJSON')

In [ ]:
# Set shp_in and geojson_out
shp_in = data_dir + '/BTS_DESA_NASIONAL_V21_0.shp'
geojson_out = out_dir + '/BTS_DESA_NASIONAL_V21_0.json'

load_shapefile_into_geojson(shp_in, geojson_out)